In [1]:
# Name: data_generator.ipynb
# Authors: Stephan Meighen-Berger
# Used to create a statistics dataset

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
for seed in range(100):
    # Module imports
    from fourth_day import Fourth_Day, config
    # ---------------------------------------------
    # Some example settings
    # For a full lost check the config file
    # ---------------------------------------------
    # Scenario Settings
    # These are general settings pertaining to the simulation run
    config["general"]["random state seed"] = seed
    config['scenario']['population size'] = 10  # The starting population size
    config['scenario']['duration'] = 600 * 1  # Total simulation time in seconds
    config['scenario']['exclusion'] = True  # If an exclusion zone should be used (the detector)
    config['scenario']['injection']['rate'] = 1e0  #  Injection rate in per second, a distribution is constructed from this value
    config['scenario']['injection']['y range'] = [0., 15.]  # The y-range of injection
    config['scenario']['light prop'] = {  # Where the emitted light should be propagated to (typically the detector location)
                "switch": True,  # If light should be propagated
                "x_pos": 3.,  # The x-coordinates
                "y_pos": 0.5 * 15. - 0.15,  # The y-coordinates
            }
    config['scenario']['detector'] = {  # detector specific properties, positions are defined as offsets from the light prop values
        "switch": True,  # If the detector should be modelled
        "type": "PMTSpec",  # Detector name, implemented types are given in the config
        "response": True,  # If a detector response should be used
        "acceptance": "Flat",  # Flat acceptance
        "mean detection prob": 0.5  # Used for the acceptance calculation
    }
    # ---------------------------------------------
    # Organisms
    # Organisms properties are defined here
    config['organisms']['emission fraction'] = 0.2  # Amount of energy an organism uses per pulse
    config['organisms']['alpha'] = 1e0  # Proportionality factor for the emission probability
    config['organisms']["minimal shear stress"] = 0.005  # The minimal amount of shear stress needed to emit (generic units)
    config["organisms"]["filter"] = 'depth'  # Method of filtering organisms (here depth)
    config["organisms"]["depth filter"] = 10000.  # Organisms need to exist below this depth
    # ---------------------------------------------
    # Geometry
    # These settings define the geometry of the system
    # Typically a box (simulation volume) with a spherical exclusion zone (detector)
    config['geometry']['volume'] = {
        'function': 'rectangle',
        'x_length': 30.,
        'y_length': 15.,
        'offset': None,
    }
    # Reduce the observation size to reduce the computational load
    config['geometry']['observation'] = {
        'function': 'rectangle',
        'x_length': 30.,
        'y_length': 15.,
        "offset": np.array([0., 0.]),
    }
    # The detector volume
    config['geometry']["exclusion"] = {
        "function": "sphere",
        "radius": 0.15,
        "x_pos": 3.,
        "y_pos": 0.5 * 15. - 0.15,
    }
    # ---------------------------------------------
    # Water
    # Properties of the current model
    config['water']['model']['name'] = 'custom'  # Use a custom (non analytic) model
    config['water']['model']['off set'] = np.array([0., 0.])  # Offset of the custom model
    config['water']['model']['directory'] = "../data/current/Parabola_5mm/run_10cm_npy/"  # The files used by the custom model
    config['water']['model']['time step'] = 1.  # in Seconds
    # Creating a fourth_day object
    # Use this to interact with with the module
    fd = Fourth_Day()
    # Launching solver
    # This might take a while depending on the settings
    # Check the log file (standard setting is in the run folder) for the current iteration
    fd.sim()
    pickle.dump(fd.statistics[301:], open('storage/statistics_seed_%d.p' %seed, "wb"))